<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Language_modeling/N_Gram_language_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#N-gram language models

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk

from nltk.tokenize import WordPunctTokenizer

In [2]:
%matplotlib inline

## Load data

In [3]:
%%capture
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz

In [4]:
data = pd.read_json("./arxivData.json")

## EDA

In [5]:
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
14252,"[{'name': 'Yuxin Chen'}, {'name': 'Yuejie Chi'...",2,1310.0807v5,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,Statistical inference and information processi...,"[{'term': 'cs.IT', 'scheme': 'http://arxiv.org...",Exact and Stable Covariance Estimation from Qu...,2013
16430,"[{'name': 'Xiaochang Peng'}, {'name': 'Chuan W...",16,1702.05053v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Neural attention models have achieved great su...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Addressing the Data Sparsity Issue in Neural A...,2017
18348,"[{'name': 'Youssef Hamadi'}, {'name': 'Said Ja...",31,0904.0029v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,In this paper a new dynamic subsumption techni...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Learning for Dynamic subsumption,2009
11529,"[{'name': 'James Newling'}, {'name': 'François...",23,1605.06950v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,"We present a new algorithm, trimed, for obtain...","[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",A Sub-Quadratic Exact Medoid Algorithm,2016
39143,"[{'name': 'Canyi Lu'}, {'name': 'Yunchao Wei'}...",28,1404.6871v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,This paper proposes the Proximal Iteratively R...,"[{'term': 'cs.NA', 'scheme': 'http://arxiv.org...",Proximal Iteratively Reweighted Algorithm with...,2014


In [6]:
data.shape

(41000, 9)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41000 entries, 0 to 40999
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   41000 non-null  object
 1   day      41000 non-null  int64 
 2   id       41000 non-null  object
 3   link     41000 non-null  object
 4   month    41000 non-null  int64 
 5   summary  41000 non-null  object
 6   tag      41000 non-null  object
 7   title    41000 non-null  object
 8   year     41000 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 2.8+ MB


In [8]:
for col in data.columns:
  print('Unique:', col, data[col].nunique())
  if data[col].dtype == 'int64':
    print(f"{'_'*5}", col)
    display(data[col].describe())
  print()

Unique: author 36157

Unique: day 31
_____ day


count    41000.000000
mean        15.833293
std          8.699371
min          1.000000
25%          8.000000
50%         16.000000
75%         23.000000
max         31.000000
Name: day, dtype: float64


Unique: id 41000

Unique: link 41000

Unique: month 12
_____ month


count    41000.000000
mean         6.547488
std          3.525254
min          1.000000
25%          3.000000
50%          6.000000
75%         10.000000
max         12.000000
Name: month, dtype: float64


Unique: summary 40967

Unique: tag 5874

Unique: title 40955

Unique: year 26
_____ year


count    41000.000000
mean      2014.484073
std          3.685414
min       1993.000000
25%       2013.000000
50%       2016.000000
75%       2017.000000
max       2018.000000
Name: year, dtype: float64

### Duplicates

In [9]:
data[data.duplicated(subset=['title', 'summary'], keep='first')]

,author,day,id,link,month,summary,tag,title,year
10066,"[{'name': 'Xinghao Pan'}, {'name': 'Jianmin Ch...",19,1702.05800v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Distributed training of deep learning models o...,"[{'term': 'cs.DC', 'scheme': 'http://arxiv.org...",Revisiting Distributed Synchronous SGD,2017
10261,[{'name': 'Daniil Ryabko'}],5,1005.0826v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,"The problem of clustering is considered, for t...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Clustering processes,2010
10824,[{'name': 'David Wipf'}],9,1408.2054v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",8,In many applications that require matrix solut...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Non-Convex Rank Minimization via an Empirical ...,2014
10826,"[{'name': 'Jie Chen'}, {'name': 'Nannan Cao'},...",9,1408.2060v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",8,Gaussian processes (GP) are Bayesian non-param...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Parallel Gaussian Process Regression with Low-...,2014
10844,"[{'name': 'Maren Mahsereci'}, {'name': 'Philip...",10,1502.02846v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,"In deterministic optimization, line searches a...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Probabilistic Line Searches for Stochastic Opt...,2015
10944,"[{'name': 'Steffen Lauritzen'}, {'name': 'Nico...",5,1211.0806v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,"Discussion of ""Latent variable graphical model...","[{'term': 'math.ST', 'scheme': 'http://arxiv.o...",Discussion: Latent variable graphical model se...,2012
10945,[{'name': 'Martin J. Wainwright'}],5,1211.0808v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,"Discussion of ""Latent variable graphical model...","[{'term': 'math.ST', 'scheme': 'http://arxiv.o...",Discussion: Latent variable graphical model se...,2012
10946,"[{'name': 'Emmanuel J. Candés'}, {'name': 'Mah...",5,1211.0817v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,"Discussion of ""Latent variable graphical model...","[{'term': 'math.ST', 'scheme': 'http://arxiv.o...",Discussion: Latent variable graphical model se...,2012
15413,"[{'name': 'Daniel Fried'}, {'name': 'Kevin Duh'}]",18,1412.5836v3,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,We investigate the hypothesis that word repres...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Incorporating Both Distributional and Relation...,2014
20087,"[{'name': 'Peter D. Grunwald'}, {'name': 'Jose...",27,1407.7188v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,It is commonly-accepted wisdom that more infor...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",When Ignorance is Bliss,2014


## Graphs

In [10]:
data

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017
...,...,...,...,...,...,...,...,...,...
40995,"[{'name': 'Vitaly Feldman'}, {'name': 'Pravesh...",18,1404.4702v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,We study the complexity of learning and approx...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Nearly Tight Bounds on $\ell_1$ Approximation ...,2014
40996,"[{'name': 'Orly Avner'}, {'name': 'Shie Mannor'}]",22,1404.5421v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,We consider the problem of multiple users targ...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Concurrent bandits and cognitive radio networks,2014
40997,"[{'name': 'Ran Zhao'}, {'name': 'Deanna Needel...",22,1404.5899v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,"In this paper, we compare and analyze clusteri...","[{'term': 'math.NA', 'scheme': 'http://arxiv.o...",A Comparison of Clustering and Missing Data Me...,2014
40998,"[{'name': 'Zongyan Huang'}, {'name': 'Matthew ...",25,1404.6369v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",4,Cylindrical algebraic decomposition(CAD) is a ...,"[{'term': 'cs.SC', 'scheme': 'http://arxiv.org...",Applying machine learning to the problem of ch...,2014


### Assemble lines: concatenate title and description

In [11]:
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()
sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

## Tokenization

In [12]:
wp_tokenizer = WordPunctTokenizer()

In [13]:
wpt_lines = [' '.join(wp_tokenizer.tokenize(x.lower())) for x in lines]

In [14]:
assert sorted(wpt_lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(wpt_lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

# N-Gram language Models explanation

Language model is probabilistic and estimates text probability: of all tokens from  $w_1$ to $w_t$ in text X:
$$P(X) = P(w_1, ..., w_t)$$

It is done by using the chain rule we all love:


$$P(w_1, .., w_t) = P(w_1) \times P(w_2|w_1) \times P(w_3|w_1, w_2) \times P(w_t|w_1,..., w_t)$$

The problem is that for **N** words in a sequence the final term $P(w_t|w_1, ..., w_{t-1})$ depends on probabilities of **N-1** previous words.

For longer texts we can use N-grams where *next word depends* only on the **n** amount of previous words (usually 2 or 3) so the **probability** of the **last term** would be:

$$P(w_t|w_1, .., w_t-1) = P(w_t|w_{t-n+1},..., w_{t-1})$$

and the whole **text approximation** (n-th order Markov assumption) would be:

$$P(w_1, .., w_N) = \underset{t}∏ P(w_t|w_{t-n+1},..., w_{t-1})$$
